In [185]:
import pandas as pd

In [187]:
#dataset read the csv and use ; to separate it
dataset=pd.read_csv("Data/airfoil_self_noise.dat",sep="\t",index_col=False,names=["Frequency","Angle","Chord","Velocity","Suction","Pressure Level"], header = None)

In [188]:
#20% dataset will be test dataset, the rest of it will be train dataset
from sklearn.model_selection import train_test_split
train, test=train_test_split(dataset,test_size=0.2,random_state=1)

In [189]:
#Pressure Level don't need to be normalized
#so drop it before normalization
x_train = train.drop("Pressure Level", axis = 1)
y_train = train["Pressure Level"]
x_test = test.drop("Pressure Level", axis = 1)
y_test = test["Pressure Level"]

In [190]:
#set x_train to norm then build model
#after building model, normalize x_train and x_test
from sklearn.preprocessing import Normalizer
norm = Normalizer()
norm.fit(x_train)
n_x_train = norm.transform(x_train)  #normalized train dataset
n_x_test = norm.transform(x_test) #normalized test dataset

In [191]:
#feature selection, k = 3 means the number of features will decrease to 3
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif,k=3)  #the reason why use f_classif is that chi2 can't use float
s_n_x_train = selector.fit_transform(n_x_train, y_train)
s_n_x_test = selector.transform(n_x_test)

In [192]:
#build svm regression model
from sklearn.svm import SVR

# svr=SVR(C=1.0,kernel="poly",degree=3,gamma='auto')
# svr=SVR(kernel="poly", degree=3, C=1, epsilon=0.1, gamma="auto")
svr = SVR()
#different parameters of models

svr.fit(s_n_x_train,y_train)

SVR()

In [193]:
#if don't normalization, the result will very bad

In [194]:
#the accuary of regression score
regression_score=svr.score(s_n_x_test,y_test)
print('The regression accuracy score is {:03.2f}'.format(regression_score))
regression_score

The regression accuracy score is 0.16


0.15748051753448666

In [195]:
#the result of k_fold
from sklearn.model_selection import cross_val_score
y_predict=svr.predict(s_n_x_test)
cross_val_score(svr, s_n_x_train, y_train, cv=5, scoring='neg_mean_squared_error')

array([-35.16303006, -37.92591314, -31.94268394, -40.34971488,
       -38.17845231])